In [43]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import os
from weather import parse_weather_data
from odds import get_metabet_spread
import requests
from nfl_teams import NFL_TEAM_MAP

# Example data url: https://www.pro-football-reference.com/players/D/DiggSt00.htm

SEASON_START = '08/26/2025'

def get_week_relative_to_start(season_start=SEASON_START):
    """Calculate current week relative to season start date."""
    start = datetime.strptime(season_start, '%m/%d/%Y')
    today = datetime.today()
    num_days = (today - start).days + 1
    return math.ceil(num_days / 7)

def american_odds_to_probability(odds):
    """
    Convert American odds to implied probability.
    
    Args:
        odds: American odds (e.g., +195, -120)
    
    Returns:
        float: Implied probability (0-1)
    """
    if odds > 0:
        return 100 / (odds + 100)
    else:
        return abs(odds) / (abs(odds) + 100)

def probability_to_american_odds(probability):
    """
    Convert probability to American odds.
    
    Args:
        probability: Probability (0-1)
    
    Returns:
        int: American odds
    """
    if probability > 0.5:
        return int(-probability / (1 - probability) * 100)
    else:
        return int((1 - probability) / probability * 100)

WEEK = max(get_week_relative_to_start(), 1)

In [44]:

class PropBet:
    """Represents a prop bet with its parameters."""
    
    def __init__(self, prop_type, value):
        """
        Initialize a prop bet.
        
        Args:
            prop_type: Type of prop ('anytime_td', 'yards_over_under', 'yards_or_above', etc.)
            value: The line value or odds (e.g., 0.5, +195, -113, 51)
        """
        self.prop_type = prop_type
        self.value = value
        
        # Auto-generate description based on prop type and value
        if prop_type == 'anytime_td':
            self.description = f"Anytime TD {value:+d}"
        elif prop_type == 'yards_over_under':
            self.description = f"Yards Over/Under {value}"
        elif prop_type == 'yards_or_above':
            yards_threshold, odds = value
            self.description = f"{yards_threshold} Yards or Above {odds:+d}"
        elif prop_type == 'touchdowns_over_under':
            self.description = f"Touchdowns Over/Under {value}"
        else:
            self.description = f"{prop_type}: {value}"
    
    def __repr__(self):
        return f"PropBet({self.prop_type}, {self.value})"


In [45]:

class PropPredictor:
    """
    Unified prop predictor that analyzes player props using:
    - Historical player stats
    - Weather conditions
    - Betting spreads
    - Game totals
    """
    
    def __init__(self, player_csv_path, team_code, position, opponent=None, week=None):
        """
        Initialize predictor with player stats and optional game context.
        
        Args:
            player_csv_path: Path to player stats CSV (e.g., 'playerstats/ne-wr.csv')
            team_code: Team abbreviation (e.g., 'NE')
            position: Player position (e.g., 'WR')
            opponent: Opponent team code (e.g., 'LAC') - optional
            week: Week number for fetching current spread/weather - optional
        """
        self.player_csv_path = player_csv_path
        self.team_code = team_code
        self.position = position
        self.opponent = opponent
        self.week = week
        self.stats_df = pd.read_csv(player_csv_path)
        self.stats_df['Date'] = pd.to_datetime(self.stats_df['Date'])
        self.spread_data = None
        self.weather_data = None
        
        if week:
            self.load_game_data()
        
    def load_game_data(self):
        """Load spread data for the current week."""
        try:
            self.spread_data = get_metabet_spread(self.week)
            print(f"✓ Loaded spread data for week {self.week}")
        except Exception as e:
            print(f"⚠ Could not load spread data: {e}")
            self.spread_data = pd.DataFrame()
    
    def get_recent_average(self, stat_column, games=5):
        """Calculate average of a stat over recent games."""
        recent = self.stats_df.head(games)
        return recent[stat_column].mean()
    
    def get_advanced_weather_impact(self):
        """
        Get advanced weather impact with nuanced calculations.
        
        Returns:
            dict with detailed weather impacts
        """
        impact = {
            'wind': 1.0,
            'precipitation': 1.0,
            'temperature': 1.0,
            'dome': False,
            'total': 1.0
        }
        
        if self.weather_data is None or self.weather_data.empty:
            return impact
        
        # Wind adjustments for WR (passing game affected)
        if 'wind' in self.weather_data.columns:
            wind_speed = self.weather_data['wind'].iloc[0]
            if isinstance(wind_speed, (int, float)):
                if wind_speed > 20:
                    impact['wind'] = 0.85
                elif wind_speed > 15:
                    impact['wind'] = 0.90
                elif wind_speed > 10:
                    impact['wind'] = 0.95
        
        # Precipitation reduces passing accuracy
        if 'precipitation' in self.weather_data.columns:
            precip = self.weather_data['precipitation'].iloc[0]
            if pd.notna(precip) and precip > 0:
                if precip > 0.5:
                    impact['precipitation'] = 0.80
                else:
                    impact['precipitation'] = 0.90
        
        # Dome games have no weather impact
        if 'dome' in self.weather_data.columns and self.weather_data['dome'].iloc[0]:
            impact['dome'] = True
            impact['wind'] = 1.0
            impact['precipitation'] = 1.0
        
        impact['total'] = impact['wind'] * impact['precipitation'] * impact['temperature']
        return impact
    
    def get_advanced_spread_impact(self):
        """
        Get advanced spread impact analysis.
        
        Returns:
            dict with spread details and impact
        """
        impact = {
            'team_spread': None,
            'is_favorite': None,
            'spread_multiplier': 1.0,
            'game_total': None,
            'pace_multiplier': 1.0,
            'final_multiplier': 1.0
        }
        
        if self.spread_data is None or self.spread_data.empty:
            return impact
        
        try:
            # Find team's spread in the data
            for _, row in self.spread_data.iterrows():
                away = str(row.get('AwayTeam', '')).upper().strip()
                home = str(row.get('HomeTeam', '')).upper().strip()
                
                if self.team_code in away or self.team_code in home:
                    try:
                        spread = float(row.get('PointSpread', 0))
                        
                        impact['team_spread'] = spread
                        impact['is_favorite'] = spread < 0
                        
                        # Favorites get more volume
                        if spread < -3:
                            impact['spread_multiplier'] = 1.08
                        elif spread < 0:
                            impact['spread_multiplier'] = 1.04
                        elif spread > 3:
                            impact['spread_multiplier'] = 0.92
                        else:
                            impact['spread_multiplier'] = 0.96
                        
                        # Game total impact
                        total = float(row.get('OverUnder', 0))
                        if total > 0:
                            impact['game_total'] = total
                            # High-scoring games = more volume
                            if total > 50:
                                impact['pace_multiplier'] = 1.05
                            elif total < 40:
                                impact['pace_multiplier'] = 0.95
                        
                        break
                    except Exception as e:
                        continue
        except Exception as e:
            pass
        
        impact['final_multiplier'] = impact['spread_multiplier'] * impact['pace_multiplier']
        return impact
    
    def predict_yards(self, prop_line, season_or_week='current'):
        """
        Predict receiving/rushing yards with optional weather/spread adjustments.
        
        Args:
            prop_line: The over/under line (e.g., 65.5)
            season_or_week: Optional week identifier for spread data
            
        Returns:
            dict with prediction and analysis
        """
        # Calculate average yards
        if self.position == 'WR' or self.position == 'TE':
            yards_col = 'Yds'  # Receiving yards
        else:
            yards_col = 'Yds'
        
        recent_avg = self.get_recent_average(yards_col, games=5)
        overall_avg = self.stats_df[yards_col].mean()
        
        # Get adjustments
        weather_impact = self.get_advanced_weather_impact()
        spread_impact = self.get_advanced_spread_impact()
        
        # Adjusted projection
        adjusted_projection = recent_avg
        adjusted_projection *= weather_impact['total']
        adjusted_projection *= spread_impact['final_multiplier']
        
        # Calculate edge
        margin = adjusted_projection - prop_line
        edge_percent = (margin / prop_line * 100) if prop_line > 0 else 0
        
        if abs(edge_percent) < 3:
            confidence = 'LOW'
            rec = 'PASS'
        elif edge_percent > 7:
            confidence = 'HIGH' if edge_percent > 12 else 'MEDIUM'
            rec = 'OVER'
        elif edge_percent < -7:
            confidence = 'HIGH' if edge_percent < -12 else 'MEDIUM'
            rec = 'UNDER'
        else:
            confidence = 'MEDIUM'
            rec = 'PASS' if abs(edge_percent) < 5 else ('OVER' if edge_percent > 0 else 'UNDER')
        
        result = {
            'prop_line': prop_line,
            'recent_avg': round(recent_avg, 2),
            'overall_avg': round(overall_avg, 2),
            'projected_yards': round(adjusted_projection, 2),
            'recommendation': rec,
            'confidence': confidence,
            'edge_percent': round(edge_percent, 1),
            'weather_details': weather_impact,
            'spread_details': spread_impact,
            'analysis': f"Projected {round(adjusted_projection, 1)} yards vs line of {prop_line}. {confidence} confidence {rec}."
        }
        
        return result
    
    def predict_touchdowns(self, prop_line):
        """
        Predict touchdown probability.
        
        Args:
            prop_line: The over/under line (e.g., 0.5)
            
        Returns:
            dict with prediction and analysis
        """
        recent_avg = self.get_recent_average('TD', games=5)
        overall_avg = self.stats_df['TD'].mean()
        
        # Touchdowns are less predictable, but use trend
        projected_td = recent_avg
        
        result = {
            'prop_line': prop_line,
            'recent_avg': round(recent_avg, 2),
            'overall_avg': round(overall_avg, 2),
            'projected_td': round(projected_td, 2),
            'recommendation': None,
            'confidence': None,
            'analysis': None
        }
        
        # TD predictions are inherently uncertain
        margin = projected_td - prop_line
        
        if abs(margin) < 0.2:
            confidence = 'LOW'
            rec = 'PASS'
        elif margin > 0.3:
            confidence = 'MEDIUM'
            rec = 'OVER'
        elif margin < -0.3:
            confidence = 'MEDIUM'
            rec = 'UNDER'
        else:
            confidence = 'LOW'
            rec = 'PASS'
        
        result['recommendation'] = rec
        result['confidence'] = confidence
        result['analysis'] = f"Projected {projected_td} TDs vs line of {prop_line}. {confidence} confidence {rec}."
        
        return result
    
    def predict_anytime_td(self, american_odds):
        """
        Predict whether anytime TD odds are a good deal.
        
        Args:
            american_odds: American odds (e.g., +195, -120)
            
        Returns:
            dict with prediction, implied probability, and EV analysis
        """
        # Calculate TD hit rate (games with TD / total games) - more realistic probability
        recent_games = self.stats_df.head(5)
        recent_games_with_td = (recent_games['TD'] > 0).sum()
        recent_td_hit_rate = recent_games_with_td / len(recent_games) if len(recent_games) > 0 else 0
        
        overall_games = self.stats_df
        overall_games_with_td = (overall_games['TD'] > 0).sum()
        overall_td_hit_rate = overall_games_with_td / len(overall_games) if len(overall_games) > 0 else 0
        
        # Blend recent and season average (regression to mean)
        # 60% recent, 40% season to avoid overreacting to small samples
        our_probability = (recent_td_hit_rate * 0.6) + (overall_td_hit_rate * 0.4)
        
        # Vegas implied probability
        implied_prob = american_odds_to_probability(american_odds)
        
        # Calculate payout and EV
        bet_amount = 100
        if american_odds > 0:
            profit = bet_amount * american_odds / 100
            total_payout = bet_amount + profit
        else:
            profit = bet_amount * 100 / abs(american_odds)
            total_payout = bet_amount + profit
        
        # Expected value per $100 bet
        # EV = (Prob_Win * Profit) - (Prob_Loss * Bet)
        ev = (our_probability * profit) - ((1 - our_probability) * bet_amount)
        ev_percent = (ev / bet_amount) * 100
        
        # Recommendation based on edge
        if our_probability > implied_prob + 0.05:
            rec = 'BET'
            confidence = 'HIGH' if our_probability > implied_prob + 0.10 else 'MEDIUM'
        elif our_probability > implied_prob + 0.02:
            rec = 'LEAN'
            confidence = 'MEDIUM'
        elif our_probability > implied_prob:
            rec = 'PASS'
            confidence = 'LOW'
        else:
            rec = 'AVOID'
            confidence = 'MEDIUM' if implied_prob - our_probability > 0.05 else 'LOW'
        
        result = {
            'american_odds': american_odds,
            'implied_probability': round(implied_prob, 4),
            'our_probability': round(our_probability, 4),
            'recent_hit_rate': round(recent_td_hit_rate, 4),
            'overall_hit_rate': round(overall_td_hit_rate, 4),
            'profit_if_win': round(profit, 2),
            'expected_value': round(ev, 2),
            'ev_percent': round(ev_percent, 2),
            'recommendation': rec,
            'confidence': confidence,
            'analysis': f"Vegas implies {implied_prob*100:.1f}% probability. We project {our_probability*100:.1f}%. EV: ${ev:.2f} per $100 bet ({ev_percent:.1f}%)."
        }
        
        return result
    
    def predict_yards_or_above(self, yards_threshold, american_odds):
        """
        Predict whether "yards or above" prop is a good deal.
        
        Args:
            yards_threshold: Minimum yards for prop to hit (e.g., 51)
            american_odds: American odds (e.g., -113)
            
        Returns:
            dict with prediction and EV analysis
        """
        recent_avg = self.get_recent_average('Yds', games=5)
        overall_avg = self.stats_df['Yds'].mean()
        
        # Estimate probability of hitting threshold
        # Using recent performance as baseline
        recent_games = self.stats_df.head(5)
        games_above = (recent_games['Yds'] >= yards_threshold).sum()
        our_probability = games_above / len(recent_games) if len(recent_games) > 0 else 0.5
        
        # Vegas implied probability
        implied_prob = american_odds_to_probability(american_odds)
        
        # Calculate payout and EV
        bet_amount = 100
        if american_odds > 0:
            profit = bet_amount * american_odds / 100
        else:
            profit = bet_amount * 100 / abs(american_odds)
        
        # Expected value per $100 bet
        ev = (our_probability * profit) - ((1 - our_probability) * bet_amount)
        ev_percent = (ev / bet_amount) * 100
        
        # Recommendation based on edge
        if our_probability > implied_prob + 0.05:
            rec = 'BET'
            confidence = 'HIGH' if our_probability > implied_prob + 0.10 else 'MEDIUM'
        elif our_probability > implied_prob + 0.02:
            rec = 'LEAN'
            confidence = 'MEDIUM'
        elif our_probability > implied_prob:
            rec = 'PASS'
            confidence = 'LOW'
        else:
            rec = 'AVOID'
            confidence = 'MEDIUM' if implied_prob - our_probability > 0.05 else 'LOW'
        
        result = {
            'yards_threshold': yards_threshold,
            'american_odds': american_odds,
            'implied_probability': round(implied_prob, 4),
            'our_probability': round(our_probability, 4),
            'recent_avg': round(recent_avg, 2),
            'overall_avg': round(overall_avg, 2),
            'games_above_in_recent': games_above,
            'expected_value': round(ev, 2),
            'ev_percent': round(ev_percent, 2),
            'recommendation': rec,
            'confidence': confidence,
            'analysis': f"Vegas implies {implied_prob*100:.1f}% hit rate. We project {our_probability*100:.1f}% ({games_above}/5 recent games). EV: ${ev:.2f} per $100 bet ({ev_percent:.1f}%)."
        }
        
        return result
    
    def predict_prop(self, prop_bet):
        """
        Unified method to predict any prop type.
        
        Args:
            prop_bet: PropBet instance with prop_type and value
            
        Returns:
            dict with prediction and analysis
        """
        if prop_bet.prop_type == 'anytime_td':
            return self.predict_anytime_td(prop_bet.value)
        elif prop_bet.prop_type == 'yards_over_under':
            return self.predict_yards(prop_bet.value)
        elif prop_bet.prop_type == 'touchdowns_over_under':
            return self.predict_touchdowns(prop_bet.value)
        elif prop_bet.prop_type == 'yards_or_above':
            # For this type, value should be (yards_threshold, american_odds)
            if isinstance(prop_bet.value, tuple) and len(prop_bet.value) == 2:
                return self.predict_yards_or_above(prop_bet.value[0], prop_bet.value[1])
            else:
                raise ValueError("yards_or_above requires value as (threshold, odds) tuple")
        else:
            raise ValueError(f"Unknown prop type: {prop_bet.prop_type}")
    
    def print_player_summary(self):
        """Print recent performance summary."""
        print(f"\n{'='*60}")
        print(f"Player Stats Summary - {self.team_code} {self.position}")
        print(f"{'='*60}")
        
        recent = self.stats_df.head(5)
        print("\nLast 5 Games:")
        
        # Determine which columns to display based on position
        if self.position == 'RB':
            display_cols = ['Date', 'Team', 'Opp', 'Result', 'Att', 'Yds', 'TD', 'Rec']
            yards_label = "Rushing Yards"
            carries_label = "Carries"
        else:  # WR, TE, or default
            display_cols = ['Date', 'Team', 'Opp', 'Result', 'Rec', 'Yds', 'TD', 'Tgt']
            yards_label = "Receiving Yards"
            carries_label = "Receptions"
        
        # Only show columns that exist in the dataframe
        available_cols = [col for col in display_cols if col in recent.columns]
        print(recent[available_cols].to_string(index=False))
        
        print(f"\nRecent Averages (Last 5 Games):")
        
        # For RB: show rushing yards from first Yds column, uses Att for carries
        if self.position == 'RB':
            # Use the first Yds column for rushing
            rushing_yds = recent.iloc[:, recent.columns.get_loc('Yds')].mean() if 'Yds' in recent.columns else 0
            print(f"  {yards_label}: {rushing_yds:.1f}")
            if 'Att' in self.stats_df.columns:
                print(f"  {carries_label}: {self.get_recent_average('Att', 5):.1f}")
        else:
            # For WR: show receiving yards
            print(f"  {yards_label}: {self.get_recent_average('Yds', 5):.1f}")
            if 'Rec' in self.stats_df.columns:
                print(f"  {carries_label}: {self.get_recent_average('Rec', 5):.1f}")
        
        print(f"  Touchdowns: {self.get_recent_average('TD', 5):.2f}")
        
        print(f"\nSeason Averages:")
        if self.position == 'RB':
            rushing_yds_season = self.stats_df.iloc[:, self.stats_df.columns.get_loc('Yds')].mean() if 'Yds' in self.stats_df.columns else 0
            print(f"  {yards_label}: {rushing_yds_season:.1f}")
            if 'Att' in self.stats_df.columns:
                print(f"  {carries_label}: {self.stats_df['Att'].mean():.1f}")
        else:
            print(f"  {yards_label}: {self.stats_df['Yds'].mean():.1f}")
            if 'Rec' in self.stats_df.columns:
                print(f"  {carries_label}: {self.stats_df['Rec'].mean():.1f}")
        
        print(f"  Touchdowns: {self.stats_df['TD'].mean():.2f}")
    
    def print_detailed_analysis(self, prop_line, prop_type='yards'):
        """Print detailed analysis including all adjustments."""
        
        print(f"\n{'='*70}")
        print(f"DETAILED PROP ANALYSIS - {prop_type.upper()}")
        print(f"{'='*70}")
        
        if prop_type == 'yards':
            result = self.predict_yards(prop_line)
        elif prop_type == 'anytime_td':
            result = self.predict_anytime_td(prop_line)
        else:
            result = self.predict_touchdowns(prop_line)
        
        print(f"\nBASE STATS:")
        print(f"  Recent Avg (5 games): {result['recent_avg']}")
        print(f"  Season Avg: {result['overall_avg']}")
        
        if 'weather_details' in result:
            weather = result.get('weather_details', {})
            if weather:
                print(f"\nWEATHER IMPACT:")
                print(f"  Dome Game: {'Yes' if weather.get('dome') else 'No'}")
                print(f"  Wind Impact: {weather.get('wind', 1.0):.2f}x")
                print(f"  Precipitation Impact: {weather.get('precipitation', 1.0):.2f}x")
                print(f"  Total Weather Multiplier: {weather.get('total', 1.0):.3f}x")
            
            spread = result.get('spread_details', {})
            if spread and spread.get('team_spread') is not None:
                print(f"\nSPREAD IMPACT:")
                print(f"  Team Spread: {spread.get('team_spread')} (Favorite: {spread.get('is_favorite')})")
                print(f"  Game Total: {spread.get('game_total')}")
                print(f"  Volume Multiplier: {spread.get('spread_multiplier', 1.0):.3f}x")
                print(f"  Game Pace Multiplier: {spread.get('pace_multiplier', 1.0):.3f}x")
        
        if prop_type == 'anytime_td':
            print(f"\nODDS ANALYSIS:")
            print(f"  American Odds: {result['american_odds']}")
            print(f"  Implied Probability: {result['implied_probability']*100:.2f}%")
            print(f"  Our Probability: {result['our_probability']*100:.2f}%")
            print(f"  Profit if Win (per $100): ${result['profit_if_win']:.2f}")
            
            print(f"\nEXPECTED VALUE:")
            print(f"  EV per $100 bet: ${result['expected_value']:.2f}")
            print(f"  EV %: {result['ev_percent']:.2f}%")
        else:
            print(f"\nPROJECTION:")
            print(f"  Projected: {result.get('projected_yards', result.get('projected_td'))} {prop_type}")
            print(f"  Prop Line: {prop_line}")
            print(f"  Edge: {result.get('edge_percent', 0):.1f}%")
        
        print(f"\nRECOMMENDATION:")
        print(f"  ► {result['recommendation']} ({result['confidence']} confidence)")
        print(f"  {result['analysis']}")
    
    def print_prop_analysis(self, prop_bet):
        """Print concise analysis for a PropBet instance."""
        
        result = self.predict_prop(prop_bet)
        
        print(f"\n{'='*70}")
        print(f"{prop_bet.description}")
        print(f"{'='*70}")
        
        # Handle different prop types
        if prop_bet.prop_type == 'anytime_td':
            vegas_odds = result['american_odds']
            our_prob = result['our_probability']
            fair_odds = probability_to_american_odds(our_prob)
            odds_diff = fair_odds - vegas_odds
            
            verdict = "✓ UNDERVALUED" if our_prob > result['implied_probability'] else "✗ OVERVALUED"
            
            print(f"Vegas Odds: {vegas_odds:+d}")
            print(f"Fair Odds (Our Estimate): {fair_odds:+d}")
            print(f"Difference: {odds_diff:+d}")
            print(f"EV: ${result['expected_value']:.2f} per $100 ({result['ev_percent']:.1f}%)")
            print(f"\n{verdict} - {result['recommendation'].upper()}")
        
        elif prop_bet.prop_type == 'yards_or_above':
            vegas_odds = result['american_odds']
            our_prob = result['our_probability']
            fair_odds = probability_to_american_odds(our_prob)
            odds_diff = fair_odds - vegas_odds
            
            verdict = "✓ UNDERVALUED" if our_prob > result['implied_probability'] else "✗ OVERVALUED"
            
            # Determine yard type based on position
            yard_type = "Rushing" if self.position == 'RB' else "Receiving"
            
            print(f"{result['yards_threshold']} {yard_type} Yards or Above")
            print(f"Vegas Odds: {vegas_odds:+d} ({result['implied_probability']*100:.1f}% implied)")
            print(f"Fair Odds (Our Estimate): {fair_odds:+d} ({our_prob*100:.1f}% our estimate)")
            print(f"Hit Rate (Recent): {result['games_above_in_recent']}/5 games")
            print(f"EV: ${result['expected_value']:.2f} per $100 ({result['ev_percent']:.1f}%)")
            print(f"\n{verdict} - {result['recommendation'].upper()}")
        
        else:
            print(f"Recent Avg: {result['recent_avg']} | Season Avg: {result['overall_avg']}")
            print(f"Projected: {result.get('projected_yards', result.get('projected_td'))}")
            print(f"Line: {prop_bet.value} | Edge: {result.get('edge_percent', 0):.1f}%")
            print(f"\n{result['recommendation'].upper()} ({result['confidence']})")

In [46]:

# ============================================================================
# HELPER FUNCTIONS: Dynamic CSV Loading and Player/Team Detection
# ============================================================================

import glob
from pathlib import Path

def get_latest_player_csv(playerstats_dir='playerstats'):
    """
    Get the most recently modified CSV file from playerstats folder.
    
    Returns:
        tuple: (csv_path, player_info_dict) or (None, None) if not found
    """
    csv_files = glob.glob(f"{playerstats_dir}/*.csv")
    if not csv_files:
        return None, None
    
    # Sort by modification time, most recent first
    latest_csv = max(csv_files, key=lambda f: Path(f).stat().st_mtime)
    return latest_csv, None

def extract_player_and_team_from_csv(csv_path):
    """
    Extract team and position from CSV data by analyzing columns.
    
    Args:
        csv_path: Path to player stats CSV
        
    Returns:
        dict: {'team': 'NE', 'position': 'WR', 'csv_path': csv_path}
    """
    df = pd.read_csv(csv_path)
    
    # Extract team from 'Team' column
    team = df['Team'].iloc[0].upper() if 'Team' in df.columns else 'UNKNOWN'
    
    # Infer position from column presence
    columns_lower = [col.lower() for col in df.columns]
    
    # Count rushing vs receiving columns to determine primary position
    has_att = 'att' in columns_lower
    has_rush = 'rush' in columns_lower
    has_rec = 'rec' in columns_lower
    has_tgt = 'tgt' in columns_lower
    has_cmp = 'cmp' in columns_lower
    
    # RB detection: prioritize if has rushing attempts AND more rushing-focused stats
    if (has_att or has_rush) and not has_cmp:
        # Check if this is primarily rushing data (Att column appears first or high volume)
        # RBs have rushing attempts in the Att column
        if has_att and (not has_rec or not has_tgt):
            position = 'RB'
        elif has_att and has_rec:
            # Has both - check the column order/values to determine primary position
            # If Att appears before Rec in columns, likely RB
            att_idx = columns_lower.index('att') if 'att' in columns_lower else 999
            rec_idx = columns_lower.index('rec') if 'rec' in columns_lower else 999
            if att_idx < rec_idx:
                position = 'RB'
            else:
                position = 'WR'
        else:
            position = 'RB'
    # QB detection: has pass completions
    elif has_cmp or ('att' in columns_lower and 'pass' in str(df.columns).lower()):
        position = 'QB'
    # WR/TE detection: has receptions and targets
    elif has_rec and has_tgt:
        position = 'WR'
    elif has_rec or has_tgt or 'y/r' in columns_lower:
        position = 'WR'
    # Kicker detection: has FG attempts
    elif 'fga' in columns_lower or 'fg%' in columns_lower:
        position = 'K'
    # Defenseman detection
    elif 'def' in columns_lower or 'sack' in columns_lower:
        position = 'DEF'
    else:
        position = 'UNKNOWN'
    
    return {
        'team': team,
        'position': position,
        'csv_path': csv_path
    }

def get_opponent_from_spread_data(team, spread_data):
    """
    Find opponent team from spread data given home/away team.
    
    Args:
        team: Team code (e.g., 'NE')
        spread_data: DataFrame with columns AwayTeam, HomeTeam
        
    Returns:
        str: Opponent team code, or None if not found
    """
    if spread_data is None or spread_data.empty:
        return None
    
    for _, row in spread_data.iterrows():
        away = str(row.get('AwayTeam', '')).upper().strip()
        home = str(row.get('HomeTeam', '')).upper().strip()
        
        if team in away:
            return home
        elif team in home:
            return away
    
    return None

# Execute dynamic loading
print("\n" + "="*70)
print("DYNAMIC PLAYER STATS LOADING")
print("="*70)

# Get latest CSV
latest_csv, _ = get_latest_player_csv()
if latest_csv:
    print(f"\n✓ Latest CSV found: {latest_csv}")
    
    # Extract player info
    player_info = extract_player_and_team_from_csv(latest_csv)
    team_code = player_info['team']
    position = player_info['position']
    
    print(f"  Team: {team_code}")
    print(f"  Position: {position}")
    
    # Load spread data to find opponent
    spread_data = get_metabet_spread(WEEK)
    opponent = get_opponent_from_spread_data(team_code, spread_data)
    
    print(f"  Opponent (Week {WEEK}): {opponent}")
    print(f"\n✓ Ready to analyze: {team_code} {position} vs {opponent}")
else:
    print("✗ No CSV files found in playerstats folder!")
    latest_csv = None



DYNAMIC PLAYER STATS LOADING

✓ Latest CSV found: playerstats/henderson.csv
  Team: NE
  Position: RB
return cached data week 21
  Opponent (Week 21): HOU

✓ Ready to analyze: NE RB vs HOU


In [47]:

# Initialize predictor with dynamically loaded player stats
print("\n" + "=" * 70)
print(f"PROP PREDICTION ANALYSIS - Week {WEEK}")
print("=" * 70)

if latest_csv:
    # Create predictor with loaded player info
    predictor = PropPredictor(latest_csv, team_code, position, opponent=opponent, week=WEEK)
    predictor.print_player_summary()
    
    # Define props to analyze as PropBet instances
    props_to_analyze = [
        PropBet('anytime_td', +240),
        PropBet('yards_or_above', (40, +110)),
    ]
    
    print(f"\n{'='*70}")
    print("PROP ANALYSIS")
    print(f"{'='*70}")
    
    # Analyze each prop
    for prop in props_to_analyze:
        predictor.print_prop_analysis(prop)
else:
    print("Cannot proceed - no player stats CSV file found")



PROP PREDICTION ANALYSIS - Week 21
return cached data week 21
✓ Loaded spread data for week 21

Player Stats Summary - NE RB

Last 5 Games:
      Date Team Opp  Result  Att  Yds  TD  Rec
2026-01-11   NE LAC  W 16-3    9   27   0    1
2026-01-04   NE MIA W 38-10   13   53   2    0
2025-12-28   NE NYJ W 42-10   19   82   0    0
2025-12-21   NE BAL W 28-24    5    3   0    1
2025-12-14   NE BUF L 31-35   14  148   2    2

Recent Averages (Last 5 Games):
  Rushing Yards: 62.6
  Carries: 12.0
  Touchdowns: 0.80

Season Averages:
  Rushing Yards: 52.1
  Carries: 10.5
  Touchdowns: 0.50

PROP ANALYSIS

Anytime TD +240
Vegas Odds: +240
Fair Odds (Our Estimate): +184
Difference: -56
EV: $19.38 per $100 (19.4%)

✓ UNDERVALUED - BET

40 Yards or Above +110
40 Rushing Yards or Above
Vegas Odds: +110 (47.6% implied)
Fair Odds (Our Estimate): -149 (60.0% our estimate)
Hit Rate (Recent): 3/5 games
EV: $26.00 per $100 (26.0%)

✓ UNDERVALUED - BET
